# Synthetic Validation: Why Refractive Calibration Matters

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/your-username/aquacal/blob/main/docs/tutorials/03_synthetic_validation.ipynb)

This tutorial demonstrates the importance of refractive calibration using controlled synthetic experiments. We compare refractive (n_water=1.333) vs non-refractive (n_water=1.0) calibration to show when the refractive model is essential.

**What you'll learn:**
- How non-refractive calibration introduces systematic bias
- Comparing parameter recovery accuracy between models
- Reconstruction quality differences in 3D space
- When the refractive model matters most

In [ ]:
# Data source toggle: choose rig size for synthetic experiments
RIG_SIZE = "small"  # Options: "small" (fast), "medium", "large"

# For Google Colab: uncomment to install aquacal
# !pip install aquacal

## Setup and Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from aquacal.datasets import generate_synthetic_rig
from tests.synthetic.experiment_helpers import (
    calibrate_synthetic,
    compute_per_camera_errors,
    evaluate_reconstruction,
)

print(f"Setting up {RIG_SIZE} synthetic rig...")

## Generate Ground Truth Scenario

We create a synthetic multi-camera rig with known ground truth parameters. This allows us to measure exactly how well each calibration method recovers the true geometry.

In [ ]:
# Generate synthetic scenario
scenario = generate_synthetic_rig(RIG_SIZE)

print(f"\nGround Truth Scenario: {scenario.name}")
print(f"  Cameras: {len(scenario.intrinsics)}")
print(f"  Frames: {len(scenario.board_poses)}")
print(f"  Detection noise: {scenario.noise_std} px")
print(f"  Board: {scenario.board_config.squares_x}x{scenario.board_config.squares_y}, {scenario.board_config.square_size*1000:.1f}mm squares")

# Show ground truth camera positions
camera_names = sorted(scenario.intrinsics.keys())
print(f"\nGround Truth Camera Positions (m):")
for cam_name in camera_names:
    C = scenario.extrinsics[cam_name].C
    h = scenario.interface_distances[cam_name]
    print(f"  {cam_name}: C=({C[0]:.3f}, {C[1]:.3f}, {C[2]:.3f}), interface_z={h:.3f}")

## Visualize Ground Truth Rig

In [ ]:
# 3D visualization of ground truth rig
fig = plt.figure(figsize=(12, 5))

# Top view (XY)
ax1 = fig.add_subplot(1, 2, 1)
for cam_name in camera_names:
    C = scenario.extrinsics[cam_name].C
    ax1.scatter(C[0], C[1], marker='o', s=100, label=cam_name)
    ax1.annotate(cam_name, (C[0], C[1]), xytext=(5, 5), textcoords='offset points', fontsize=8)

ax1.set_xlabel('X (m)')
ax1.set_ylabel('Y (m)')
ax1.set_title('Ground Truth: Top View (XY)')
ax1.set_aspect('equal')
ax1.grid(alpha=0.3)

# Side view (XZ)
ax2 = fig.add_subplot(1, 2, 2)
for cam_name in camera_names:
    C = scenario.extrinsics[cam_name].C
    ax2.scatter(C[0], C[2], marker='o', s=100, label=cam_name)

# Water surface
water_z = scenario.interface_distances[camera_names[0]]
ax2.axhline(water_z, color='blue', linestyle='--', alpha=0.5, linewidth=2, label='Water surface')

ax2.set_xlabel('X (m)')
ax2.set_ylabel('Z (m, down)')
ax2.set_title('Ground Truth: Side View (XZ)')
ax2.set_aspect('equal')
ax2.grid(alpha=0.3)
ax2.legend()

plt.tight_layout()
plt.show()
plt.close()

print(f"\nWater surface at Z = {water_z:.3f} m")
print(f"Cameras are in air (Z < {water_z:.3f}m), viewing underwater targets (Z > {water_z:.3f}m)")

## Experiment: Refractive vs Non-Refractive Calibration

We run calibration twice on the SAME data:
1. **Refractive model** (n_water=1.333): Correctly models Snell's law refraction
2. **Non-refractive model** (n_water=1.0): Ignores refraction (pinhole approximation)

Both optimizations minimize reprojection error, but only the refractive model can recover physically accurate parameters.

In [ ]:
print("Running refractive calibration (n_water=1.333)...")
result_refractive, detections = calibrate_synthetic(
    scenario, n_water=1.333, refine_intrinsics=True
)
errors_refractive = compute_per_camera_errors(result_refractive, scenario)

print(f"  Reprojection RMS: {result_refractive.diagnostics.reprojection_error_rms:.4f} px")
print(f"  Interface distance: {result_refractive.cameras[camera_names[0]].interface_distance:.4f} m")

print("\nRunning non-refractive calibration (n_water=1.0)...")
result_nonrefractive, _ = calibrate_synthetic(
    scenario, n_water=1.0, refine_intrinsics=True
)
errors_nonrefractive = compute_per_camera_errors(result_nonrefractive, scenario)

print(f"  Reprojection RMS: {result_nonrefractive.diagnostics.reprojection_error_rms:.4f} px")
print(f"  Interface distance: {result_nonrefractive.cameras[camera_names[0]].interface_distance:.4f} m")

print("\nBoth calibrations complete!")

## Reprojection Error Comparison

**Key insight:** Both models can achieve low reprojection error by adjusting camera parameters to fit the 2D observations. However, this doesn't guarantee correct 3D geometry!

In [ ]:
# Per-camera RMS comparison
fig, ax = plt.subplots(figsize=(10, 5))

x = np.arange(len(camera_names))
width = 0.35

rms_refr = [result_refractive.diagnostics.reprojection_error_per_camera[cam] for cam in camera_names]
rms_nonrefr = [result_nonrefractive.diagnostics.reprojection_error_per_camera[cam] for cam in camera_names]

ax.bar(x - width/2, rms_refr, width, label='Refractive (n=1.333)', color='#2196F3', alpha=0.8)
ax.bar(x + width/2, rms_nonrefr, width, label='Non-refractive (n=1.0)', color='#F44336', alpha=0.8)

ax.set_xlabel('Camera')
ax.set_ylabel('RMS Reprojection Error (px)')
ax.set_title('Reprojection Error: Refractive vs Non-Refractive')
ax.set_xticks(x)
ax.set_xticklabels(camera_names, rotation=45, ha='right')
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()
plt.close()

print(f"Overall RMS:")
print(f"  Refractive: {result_refractive.diagnostics.reprojection_error_rms:.4f} px")
print(f"  Non-refractive: {result_nonrefractive.diagnostics.reprojection_error_rms:.4f} px")
print(f"\nNote: Similar reprojection errors, but different 3D geometry!")

## Parameter Recovery Comparison

Here we see the critical difference: **refractive calibration recovers accurate physical parameters**, while non-refractive calibration introduces systematic bias.

In [ ]:
# Focal length error comparison
fig, ax = plt.subplots(figsize=(10, 5))

x = np.arange(len(camera_names))
width = 0.35

focal_refr = [errors_refractive[cam]['focal_length_error_pct'] for cam in camera_names]
focal_nonrefr = [errors_nonrefractive[cam]['focal_length_error_pct'] for cam in camera_names]

ax.bar(x - width/2, focal_refr, width, label='Refractive', color='#2196F3', alpha=0.8)
ax.bar(x + width/2, focal_nonrefr, width, label='Non-refractive', color='#F44336', alpha=0.8)

ax.set_xlabel('Camera')
ax.set_ylabel('Focal Length Error (%)')
ax.set_title('Focal Length Recovery Error vs Ground Truth')
ax.set_xticks(x)
ax.set_xticklabels(camera_names, rotation=45, ha='right')
ax.axhline(0, color='black', linewidth=0.5, linestyle='--')
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()
plt.close()

mean_focal_refr = np.mean([abs(e) for e in focal_refr])
mean_focal_nonrefr = np.mean([abs(e) for e in focal_nonrefr])

print(f"Mean absolute focal length error:")
print(f"  Refractive: {mean_focal_refr:.3f}%")
print(f"  Non-refractive: {mean_focal_nonrefr:.3f}%")
print(f"\nNon-refractive model absorbs refraction errors into focal length!")

In [ ]:
# Camera Z position error comparison
fig, ax = plt.subplots(figsize=(10, 5))

x = np.arange(len(camera_names))
width = 0.35

z_refr = [errors_refractive[cam]['z_position_error_mm'] for cam in camera_names]
z_nonrefr = [errors_nonrefractive[cam]['z_position_error_mm'] for cam in camera_names]

ax.bar(x - width/2, z_refr, width, label='Refractive', color='#2196F3', alpha=0.8)
ax.bar(x + width/2, z_nonrefr, width, label='Non-refractive', color='#F44336', alpha=0.8)

ax.set_xlabel('Camera')
ax.set_ylabel('Z Position Error (mm)')
ax.set_title('Camera Z Position Recovery Error')
ax.set_xticks(x)
ax.set_xticklabels(camera_names, rotation=45, ha='right')
ax.axhline(0, color='black', linewidth=0.5, linestyle='--')
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()
plt.close()

mean_z_refr = np.mean([abs(e) for e in z_refr])
mean_z_nonrefr = np.mean([abs(e) for e in z_nonrefr])

print(f"Mean absolute Z position error:")
print(f"  Refractive: {mean_z_refr:.2f} mm")
print(f"  Non-refractive: {mean_z_nonrefr:.2f} mm")
print(f"\n⚠️ Warning: Non-refractive parameters are systematically biased!")

## 3D Reconstruction Quality

The ultimate test: **how accurately can we triangulate 3D points?**

We measure reconstruction error by comparing triangulated board corner spacings to the known ground truth.

In [ ]:
# Load board geometry
from aquacal.core.board import BoardGeometry

board = BoardGeometry(scenario.board_config)

# Evaluate reconstruction for both models
print("Evaluating 3D reconstruction quality...")

dist_errors_refr = evaluate_reconstruction(result_refractive, board, detections)
dist_errors_nonrefr = evaluate_reconstruction(result_nonrefractive, board, detections)

print(f"\nRefractive model:")
print(f"  Signed mean error: {dist_errors_refr.signed_mean * 1000:.3f} mm")
print(f"  RMSE: {dist_errors_refr.rmse * 1000:.3f} mm")
print(f"  Measurements: {dist_errors_refr.num_comparisons}")

print(f"\nNon-refractive model:")
print(f"  Signed mean error: {dist_errors_nonrefr.signed_mean * 1000:.3f} mm")
print(f"  RMSE: {dist_errors_nonrefr.rmse * 1000:.3f} mm")
print(f"  Measurements: {dist_errors_nonrefr.num_comparisons}")

In [ ]:
# Histogram comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Refractive
if dist_errors_refr.spatial is not None:
    axes[0].hist(
        dist_errors_refr.spatial.signed_errors * 1000,
        bins=30,
        color='#2196F3',
        alpha=0.7,
        edgecolor='black'
    )
    axes[0].axvline(0, color='black', linestyle='--', alpha=0.5)
    axes[0].axvline(
        dist_errors_refr.signed_mean * 1000,
        color='red',
        linestyle='--',
        label=f'Mean: {dist_errors_refr.signed_mean * 1000:.2f} mm'
    )
    axes[0].set_xlabel('Signed Distance Error (mm)')
    axes[0].set_ylabel('Frequency')
    axes[0].set_title('Refractive Model (n=1.333)')
    axes[0].legend()
    axes[0].grid(axis='y', alpha=0.3)

# Non-refractive
if dist_errors_nonrefr.spatial is not None:
    axes[1].hist(
        dist_errors_nonrefr.spatial.signed_errors * 1000,
        bins=30,
        color='#F44336',
        alpha=0.7,
        edgecolor='black'
    )
    axes[1].axvline(0, color='black', linestyle='--', alpha=0.5)
    axes[1].axvline(
        dist_errors_nonrefr.signed_mean * 1000,
        color='red',
        linestyle='--',
        label=f'Mean: {dist_errors_nonrefr.signed_mean * 1000:.2f} mm'
    )
    axes[1].set_xlabel('Signed Distance Error (mm)')
    axes[1].set_ylabel('Frequency')
    axes[1].set_title('Non-Refractive Model (n=1.0)')
    axes[1].legend()
    axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()
plt.close()

print(f"\n3D Reconstruction RMSE ratio (non-refr / refr): {dist_errors_nonrefr.rmse / dist_errors_refr.rmse:.2f}x")
print(f"Non-refractive model has {dist_errors_nonrefr.rmse / dist_errors_refr.rmse:.1f}x larger reconstruction error!")

## When Does Refraction Matter?

**Key insight:** The importance of refractive calibration depends on the camera-to-water distance relative to the working volume depth.

- **Small interface distance** (cameras near water) → Small refraction angle → Non-refractive is "close enough"
- **Large interface distance** (cameras far from water) → Large refraction angle → Refractive model is essential

In [ ]:
# Summary comparison
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

metrics = ['focal_length_error_pct', 'z_position_error_mm', 'xy_position_error_mm']
titles = ['Focal Length Error (%)', 'Z Position Error (mm)', 'XY Position Error (mm)']

for idx, (metric, title) in enumerate(zip(metrics, titles)):
    refr_vals = [abs(errors_refractive[cam][metric]) for cam in camera_names]
    nonrefr_vals = [abs(errors_nonrefractive[cam][metric]) for cam in camera_names]
    
    mean_refr = np.mean(refr_vals)
    mean_nonrefr = np.mean(nonrefr_vals)
    
    axes[idx].bar(
        ['Refractive', 'Non-refractive'],
        [mean_refr, mean_nonrefr],
        color=['#2196F3', '#F44336'],
        alpha=0.8
    )
    axes[idx].set_ylabel('Mean Absolute Error')
    axes[idx].set_title(title)
    axes[idx].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()
plt.close()

print(f"\nInterface distance: ~{scenario.interface_distances[camera_names[0]]:.2f} m")
print(f"Working volume depth: ~{np.mean([scenario.board_poses[i][2, 3] for i in range(len(scenario.board_poses))]):.2f} m")
print(f"\nFor this geometry, refractive calibration is {'ESSENTIAL' if mean_z_nonrefr > 10 else 'recommended'}.")

## Summary

**Key Takeaways:**

1. **Both models can fit 2D observations** (low reprojection error), but only refractive calibration recovers correct 3D geometry.

2. **Non-refractive calibration introduces systematic bias** by absorbing refraction errors into focal length and camera positions.

3. **3D reconstruction error is the ultimate test**: Non-refractive models have significantly larger errors when triangulating points.

4. **Refractive calibration is essential when:**
   - Camera-to-water distance is significant (> ~0.5m)
   - High 3D accuracy is required (< 5mm)
   - Working volume extends deep underwater

5. **Non-refractive approximation may be acceptable when:**
   - Cameras are very close to water surface (< ~0.2m)
   - Only 2D tracking is needed (not 3D reconstruction)
   - Accuracy requirements are relaxed (> 10mm)

---

**Next steps:**
- Read the [theory documentation](../guide/theory.md) for mathematical details
- Try the full pipeline tutorial with real data
- Calibrate your own underwater rig!